In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.paths.directory import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.transfer.functions.transfer import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm

%env TEMP=/data/temp

/home/ipausers/bisot/pycode/MscThesis/amftrack/transfer/functions/transfer.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


env: TEMP=/data/temp


In [3]:
os.getenv("TEMP")

'/data/temp'

In [2]:
##### %autoreload 2

# directory = '/run/user/357100554/gvfs/smb-share:server=sun.amolf.nl,share=shimizu-data,user=bisot/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/'
# directory = '/media/bisot/AMF_02/PRINCE/'
directory = r"/mnt/sun-temp/TEMP/PRINCE_analysis/"
directory = "/mnt/sun/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/"

update_plate_info(directory)

# all_folders = get_current_folders(directory)
# all_folders_drop = get_current_folders("dropbox")
all_folders_drop = get_current_folders("dropbox", False)
# folders = all_folders.loc[all_folders['strain']=="Agg"]
# folders = all_folders.loc[all_folders['Plate']==734]

analysed:   0%|          | 0/2373 [00:00<?, ?it/s]

In [29]:
all_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,Temperature
30127,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_04...,862,2,'Carrot','A5','001P100N100C','Y','-',20220311,False,...,False,False,False,False,False,False,"15.03.2022, 04:23:",20220315_0423_Plate02,NaN,25
30128,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_04...,R45,39,'Carrot','A5','001P100N100C','Y','-',20220308,True,...,False,False,False,False,False,False,"15.03.2022, 04:10:",20220315_0410_Plate39,NaN,25
30129,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_04...,844,3,'Carrot','A5','001P100N100C','Y','-',20220311,False,...,False,False,False,False,False,False,"15.03.2022, 04:29:",20220315_0429_Plate03,NaN,25
30130,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_04...,865,7,'Carrot','A5','001P100N100C','Y','-',20220307,False,...,False,False,False,False,False,False,"15.03.2022, 04:35:",20220315_0435_Plate07,NaN,25
30131,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_04...,R46,38,'Carrot','A5','001P100N100C','Y','-',20220308,False,...,False,False,False,False,False,False,"15.03.2022, 04:04:",20220315_0404_Plate38,NaN,25
30132,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_04...,1007,1,'Carrot','C2','001P100N200C','Y','-',20220301,False,...,False,False,False,False,False,False,"15.03.2022, 04:16:",20220315_0416_Plate01,NaN,25
30176,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_07...,922,27,'Carrot','A5','001P100N100C','Y','-',20220304,True,...,False,False,False,False,False,False,"15.03.2022, 07:37:",20220315_0737_Plate27,NaN,25
30177,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_08...,818,34,'Carrot','A5','001P100N200C','Y','-',20220301,False,...,False,False,False,False,False,False,"15.03.2022, 08:08:",20220315_0808_Plate34,NaN,25
30178,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_07...,826,29,'Carrot','A5','001P100N100C','Y','-',20220304,False,...,False,False,False,False,False,False,"15.03.2022, 07:43:",20220315_0743_Plate29,NaN,25
30179,/mnt/sun-temp/TEMP/PRINCE_analysis/20220315_07...,R48,30,'Carrot','A5','001P100N100C','Y','-',20220308,False,...,False,False,False,False,False,False,"15.03.2022, 07:50:",20220315_0750_Plate30,NaN,25


In [16]:
directory = "/media/bisot/AMF_02/PRINCE/"
update_plate_info(directory)

analysed:   0%|          | 0/5724 [00:00<?, ?it/s]

In [17]:
all_folders = get_current_folders(directory)

In [10]:
all_folders_drop

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id
15834,/media/bisot/AMF_02/PRINCE/20191128_0623_Plate27,77,27.0,'Carrot','Agg','001P100N','Y','L','20191122',True,...,False,False,False,False,False,False,"28.11.2019, 06:23:",20191128_0623_Plate27,NaN,77_'20191122'
15833,/media/bisot/AMF_02/PRINCE/20191128_0620_Plate24,86,24.0,'Carrot','A5','100P100N','Y','R','20191103',True,...,False,False,False,False,False,False,"28.11.2019, 06:20:",20191128_0620_Plate24,NaN,86_'20191103'
15832,/media/bisot/AMF_02/PRINCE/20191128_0614_Plate21,87,21.0,'Carrot','A5','100P100N','Y','R','20191023',True,...,False,False,False,False,False,False,"28.11.2019, 06:14:",20191128_0614_Plate21,NaN,87_'20191023'
15831,/media/bisot/AMF_02/PRINCE/20191128_0611_Plate15,69,15.0,'Carrot','A5','001P100N','Y','L','20191103',True,...,False,False,False,False,False,False,"28.11.2019, 06:11:",20191128_0611_Plate15,NaN,69_'20191103'
15830,/media/bisot/AMF_02/PRINCE/20191128_0608_Plate13,31,13.0,'Carrot','A5','100P100N','Y','L','20191026',True,...,False,False,False,False,False,False,"28.11.2019, 06:08:",20191128_0608_Plate13,NaN,31_'20191026'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6305,/projects/0/einf914/data/20201123_1801_Plate39,94,39.0,Carrot,A5,001P100N,Y,NaN,20201123,True,...,True,False,True,True,True,True,"23.11.2020, 18:01:",20201123_1801_Plate39,25,9420201123
5869,/projects/0/einf914/data/20201122_1621_Plate40,69,40.0,Carrot,A5,001P100N,Y,NaN,20201119,True,...,True,False,True,True,True,False,"22.11.2020, 16:21:",20201122_1621_Plate40,25,6920201119
6299,/projects/0/einf914/data/20201121_0421_Plate40,69,40.0,Carrot,A5,001P100N,Y,NaN,20201119,True,...,True,False,True,True,True,False,"21.11.2020, 04:21:",20201121_0421_Plate40,25,6920201119
7273,/projects/0/einf914/data/20201120_1234_Plate40,69,40.0,Carrot,A5,001P100N,Y,NaN,20201119,True,...,True,False,True,True,True,False,"20.11.2020, 12:34:",20201120_1234_Plate40,25,6920201119


In [18]:
folders = all_folders

In [68]:
folders = all_folders.loc[
    all_folders["folder"].isin(all_folders_drop["folder"]) == False
]
folders2 = all_folders.loc[
    all_folders["folder"].isin(all_folders_drop["folder"]) == True
]

In [20]:
from_surf = all_folders_drop.loc[
    all_folders_drop["total_path"].str.contains("projects")
]

In [21]:
folders = all_folders.loc[all_folders["folder"].isin(from_surf["folder"]) == False]

In [97]:
folders = all_folders.loc[
    all_folders["folder"].isin(all_folders_drop["folder"]) == False
]
folders = folders.loc[folders["Temp"].isin(["25", "30to25"])]
# folders = folders.loc[folders['Temp'].isin(['25'])]

In [75]:
real_drop_folder = all_folders_drop.loc[all_folders_drop["size"] > 0.6]
folders = all_folders.loc[
    all_folders["folder"].isin(real_drop_folder["folder"]) == True
]

In [30]:
folders = all_folders.loc[all_folders["folder"].astype(str) == "20220316_0044_Plate15"]

In [78]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,Temperature
21419,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,112,14,'Carrot','A5','001P100N','Y','-',20201209,False,...,False,False,False,False,False,False,"15.11.2020, 07:59:",20201115_0759_Plate14,30to25,NaN
21498,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,90,16,'Carrot','A5','001P100N','Y','-',20201209,False,...,False,False,False,False,False,False,"11.11.2020, 16:02:",20201111_1602_Plate16,30to25,NaN
21520,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,13,12,'Carrot','A5','001P100N','Y','-',20201214,False,...,False,False,False,False,False,False,"01.01.2021, 19:23:",20210101_1923_Plate12,20to25,NaN
21541,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,13,12,'Carrot','A5','001P100N','Y','-',20201214,False,...,False,False,False,False,False,False,"28.12.2020, 23:22:",20201228_2322_Plate12,20to25,NaN
21630,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,112,14,'Carrot','A5','001P100N','Y','-',20201209,False,...,False,False,False,False,False,False,"10.11.2020, 19:56:",20201110_1956_Plate14,30to25,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29050,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,373,29,'Carrot','A5','001P100N','Y','D',20200825,True,...,True,False,True,False,False,False,"30.08.2020, 13:56:",20200830_1356_Plate29,NaN,NaN
29052,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,373,29,'Carrot','A5','001P100N','Y','D',20200825,True,...,True,False,True,False,False,False,"19.09.2020, 20:27:",20200919_2027_Plate29,NaN,NaN
29054,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,112,14,'Carrot','A5','001P100N','Y','-',20201209,False,...,False,False,False,False,False,False,"10.11.2020, 23:56:",20201110_2356_Plate14,30to25,NaN
29055,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,748,26,'Carrot','B12','100P100N','Y','-',20210917,False,...,False,False,False,False,False,False,"04.10.2021, 03:15:",20211004_0315_Plate26,NaN,NaN


In [ ]:
import shutil

run_info = folders.copy()
folder_list = list(run_info["folder"])
folder_drop_list = list(real_drop_folder["folder"].astype(str))
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list[4:]:
        directory_name = folder
        path_snap = directory + directory_name
        # print(path_snap)
        if folder in folder_drop_list:
            shutil.rmtree(path_snap)
        pbar.update(1)

transferred:   0%|          | 0/1173 [00:00<?, ?it/s]

In [46]:
folder in real_drop_folder["folder"]

False

In [51]:
folder in list(real_drop_folder["folder"].astype(str))

True

In [44]:
folders.loc[folders["folder"] == folder]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,/Img/TileConfiguration.txt.registered,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,Temperature
21409,/mnt/sun/home-folder/oyartegalvez/Drive_AMFtop...,758,17,'Carrot','Agg','001P100N','Y','-',20211007,False,...,False,False,False,False,False,False,"14.10.2021, 15:03:",20211014_1503_Plate17,NaN,NaN


In [6]:
dir_drop = "prince_data"

In [83]:
os.getenv("TEMP")

'/home/ipausers/bisot/temp'

In [68]:
run_info = folders.copy()
folder_list = list(run_info["folder"])
with tqdm(total=len(folder_list), desc="transferred") as pbar:
    for folder in folder_list:
        directory_name = folder
        run_info["unique_id"] = (
            run_info["Plate"].astype(str) + "_" + run_info["CrossDate"].astype(str)
        )
        line = run_info.loc[run_info["folder"] == directory_name]
        id_unique = line["unique_id"].iloc[0]

        path_snap = directory + directory_name
        API = str(np.load(os.getenv("HOME") + "/pycode/API_drop.npy"))

        path_zip = f'{os.getenv("TEMP")}/{directory_name}.zip'
        path_info = f'{os.getenv("TEMP")}/{directory_name}_info.json'
        line.to_json(path_info)
        zip_file(path_snap, path_zip)
        upload(
            API,
            path_zip,
            f"/{dir_drop}/{id_unique}/{directory_name}.zip",
            chunk_size=256 * 1024 * 1024,
        )
        upload(
            API,
            path_info,
            f"/{dir_drop}/{id_unique}/{directory_name}_info.json",
            chunk_size=256 * 1024 * 1024,
        )
        os.remove(path_info)
        os.remove(path_zip)
        pbar.update(1)

transferred:   0%|          | 0/18 [00:00<?, ?it/s]

zipping:   0%|          | 0/158 [00:00<?, ?it/s]

zipping:   0%|          | 0/52 [00:00<?, ?it/s]

zipping:   0%|          | 0/155 [00:00<?, ?it/s]

Uploaded:   0%|          | 0/271463393 [00:00<?, ?it/s]

...

Uploaded:   0%|          | 0/431395694 [00:00<?, ?it/s]

zipping:   0%|          | 0/152 [00:00<?, ?it/s]

zipping:   0%|          | 0/158 [00:00<?, ?it/s]

zipping:   0%|          | 0/158 [00:00<?, ?it/s]

zipping:   0%|          | 0/152 [00:00<?, ?it/s]